In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline 
from datetime import datetime, date
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder

In [ ]:
#df import with some alterations we discovered so far
df = pd.read_csv('data/xente/training.csv', parse_dates=['TransactionStartTime'], dtype={'ProductId': 'category','ProductCategory': 'category','ChannelId': 'category','PricingStrategy': 'category'}, index_col='TransactionId').drop(['CountryCode', 'CurrencyCode'], axis=1)

In [ ]:
df_var = pd.read_csv('data/xente/Xente_variable_Definitions.csv')

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df.shape

### General observations of the dataframe

In [ ]:
#set TransactionId as index, as label 
#set as categorical ProductId, ProductCategory, ChannelId
#ChannelId could be interesting 
#Investigate Amount / Value --> why is Value different sometimes
#Investigate & transform TransactionStartTime
#Random Forrest?
#convert datatypes if we deem necessary especially 'CountryCode' 

In [ ]:
#pairplot of our precleaned df

sns.pairplot(df)

In [ ]:
# our dataset is highly imbalanced --> todo: decide on how we want to address this  --> SMOTE
# possibly use Amount as predictor for basemodel --> might ignore large amount of small fraudulent transactions though

In [ ]:

#Fraud Detection: (?)

#Value of Product: (?)

#Goal : Find fraudulent transactions, save money, avoid reputation damage and prevent money laundering.

#Prediction: 'Transaction is fraudulent'
#How to handle the imbalanced dataset: SMOTE

#Evaluation Metric:
# f1-score (recommended and given by Zindi)
# Matthew Coefficient Correlation Score
# AUC - Precision-Recall Curve

#Baseline Model: (tbd)

#Score:
# f1-score = 0.1
# Matthew Coefficient Correlation Score = (tbd)
# AUC - Precision-Recall Curve = (tbd)


# ridge, lasso & elasticnet 

### We checked the unique values of different columns 

In [ ]:
df['BatchId'].unique()

In [ ]:
df['AccountId'].unique()

In [ ]:
df['SubscriptionId'].unique()

In [ ]:
#df['CurrencyCode'].unique()

In [ ]:
#only '256' --> Uganda --> no added Information --> drop this 
#df['CountryCode'].unique()

In [ ]:
#keep this for now 
df['ProductCategory'].unique()

### Groupby of to identify patterns in fraudulent transactions 

In [ ]:
df.groupby(['ProductCategory', 'FraudResult'])['FraudResult'].count()

In [ ]:
df.groupby('FraudResult')['FraudResult'].count()

### Adressing the TransactionStartTime - Column

In [ ]:
#extracting the year as a new column
df['year'] = df['TransactionStartTime'].dt.year

In [ ]:
#extracting the month as a new column
df['month'] = df['TransactionStartTime'].dt.month

In [ ]:
#extracting the date as a new column
df['TransactionStartTime_Date'] = df['TransactionStartTime'].dt.strftime('%Y-%m-%d')

In [ ]:
#extracting the starttime of the transaction 
df['TransactionStartTime_Time'] = df['TransactionStartTime'].dt.strftime('%H:%M:%S')

In [ ]:
df['TransactionStartTime_Date'] = pd.to_datetime(df['TransactionStartTime_Date'], format='%Y-%m-%d')

In [ ]:
df['TransactionStartTime_Time'] = pd.to_datetime(df['TransactionStartTime_Time'], format='%H:%M:%S').dt.time
#we need to rework the code on this to convert it to datetime 

In [ ]:
#engeneering the Weekday of the given transaction as a new column
df['weekday'] = df['TransactionStartTime_Date'].apply((lambda x: pd.Timestamp(x).day_of_week))

In [ ]:
df['weekday'].unique()

In [ ]:
df.query('weekday == 0')
#  2018-11-19 was a monday --> classified as 0
#  2019-02-11 was a monday as well --> classified as 0

In [ ]:
df.query('weekday == 6')
# 0 --> Monday
# 1 --> Tuesday
# 2 --> Wednesday
# 3 --> Thursday
# 4 --> Friday
# 5 --> Saturday
# 6 --> Sunday

In [ ]:
# How many frauds were committed during each weekday?
df.groupby(['weekday', 'FraudResult'])['FraudResult'].count()

In [ ]:
# dropping the original 'TransactionStartTime' column
df.drop('TransactionStartTime', axis=1, inplace=True)

In [ ]:
#renaming the date and time column
df.rename(columns={'TransactionStartTime_Date': 'date', "TransactionStartTime_Time": 'time'}, inplace=True)

In [ ]:
df.head()

### EDA on BatchId, AccountId, SubscriptionId, CustomerId, ProviderId, ProductId	

In [ ]:
df['BatchId'].value_counts().head(10)

In [ ]:
df.query('FraudResult == 1').value_counts(['BatchId']).head(10)

In [ ]:
df.query('FraudResult == 1').value_counts(['CustomerId']).head(10)

In [ ]:
df.query('FraudResult == 1').value_counts(['AccountId']).head(10)
#looks similiar, but one customer could have more than one Account

In [ ]:

df.query('FraudResult == 1').value_counts(['SubscriptionId']).head(10)

In [ ]:

df.query('FraudResult == 1').value_counts(['ProviderId']).head(10)

In [ ]:

df.query('FraudResult == 1').value_counts(['ProductId']).head(10)

In [ ]:
df.groupby(['ProductId', 'FraudResult'])['FraudResult'].count().sort_values()

### PyPi DataFrame Report

In [ ]:
df.profile_report()

### EDA - striping strings from categorical values and label encode them 

In [ ]:
df

In [ ]:
data = df.copy()

In [ ]:
# 0 = 'airtime' 
# 1 = 'data_bundles' 
# 2 = 'financial_services' 
# 3 = 'movies', 
# 4 = 'other'
# 5 = 'ticket', 
# 6 = 'transport', 
# 7 = 'tv', 
# 8 = 'utility_bill' 

target_label = 'ProductCategory'
le = LabelEncoder()
le.fit(df[target_label])
df['Product_new'] = le.transform(df[target_label])
df.groupby('ProductCategory')['Product_new'].mean()
df.to_csv('data/xente/Product_categories_keys.csv')

In [ ]:
# The idea is to strip away the strings from all ID columns
# BatchId	AccountId	SubscriptionId	CustomerId	ProviderId	ProductId	ChannelId
# Example: ChannelID_123 -> 123
# Scaling?